# Аннотация типов 

## Расширенный синтаксис сигнатуры функции

При определении функции в `python` тип параметров не играет роли. 
```python
def f(x):
  return x + 1
```

Тем не менее допускается его указать, или иными словами аннотировать. Например, если мы знаем, что функция `f` всегда будет работать с целочисленным аргументом, то можно объявить её так.

```python
def f(x: int):
  return x + 1
```

```{warning}
Такое объявление функции **не** значит, что её можно вызвать только с целочисленным аргументом: на этапе исполнения программы аннотация типов не используется интерпретатором `python` для проверки типов аргументов на соответствие типам параметров при вызове функции.
```

Функция `f` в примере выше очевидно вернет целочисленное значение, если ей передать целое число. Это тоже можно обозначить: для этого используется комбинация символов `->`.
```python
def f(x: int) -> int:
  return x + 1
```
```{warning}
Такое объявление функции **не** значит, что такая функция обязательно вернет целое число. На этапе исполнения программы аннотация типов не используется интерпретатором `python` для проверки типа возвращаемого значения на соответствие заявленному.
```

Пример в ячейке ниже демонстрирует, что объявленную "целочисленную" функцию `f` можно вызвать и с действительным числом.  

In [1]:
def f(x: int) -> int:
    return x + 1

x = 3.14
print(f(x))

4.140000000000001


Может показаться, что если аннотация тип не влияет на работу программы, то она бесполезна, но это вовсе не так. Сегодня при разработке программного обеспечения аннотация типов используется повсеместно. В следующем разделе приводятся три основных причины, почему аннотация типом стала стандартом индустрии. 

В рамках данного курса не удаётся поместить полноценный рассказ про аннотацию типов. Да и далеко не все научные инструменты начали аннотировать типы своих методов. Самостоятельное изучение аннотации типов можно начать [со странички в официальной документации](https://docs.python.org/3/library/typing.html). 

## Преимущества статической типизации

Динамическая типизация с одной стороны делает язык `python` чрезвычайно гибким, а с другой стороны и что-то отнимает. 

### Обнаружение ошибок  

Статическая типизация позволяет обнаруживать ошибки на этапе компиляции, т.е. до исполнения самой программы. В качестве примера рассмотрим следующий код на `C++`.

```C++
#include<iostream>
#include<string>

int f(int x){
    return x + 1;
}

int main(){
	std::string s = "abc";
	std::cout << negation(s);
}
```

Его компиляция средствами `g++` приводит к ошибке компиляции со следующим сообщением.
```
error: cannot convert 'std::__cxx11::string {aka std::__cxx11::basic_string<char>}' to 'int' for argument '1' to 'int f(int)'
  std::cout << f(s);
                  ^
```
```{figure} /_static/lecture_specific/dynamic_typing/compilation_error.gif
```


Эта ошибка связана с тем, что функция `f` объявлена с целочисленным параметром `x`, а вызывается она со строковым значением. 

Сравним это с аналогичным кодом на `python`. 
```python
def f(x):
    return x + 1

s = "abc"
print(f(s))
```

Запуск этого кода приведет к возникновению следующей ошибки. 
```
Traceback (most recent call last):
  File ".\static_typing.py", line 5, in <module>
    print(f(s))
  File ".\static_typing.py", line 2, in f
    return x + 1
TypeError: can only concatenate str (not "int") to str
```
```{figure} /_static/lecture_specific/dynamic_typing/execution_error.gif
```


Возникает ошибка при попытке сложить строку и число. Существенная разница заключается в том, что эта ошибка возникает во время исполнения программы, а не на предварительно этапе. 

Вообще говоря почти для любого языка разработаны [инструменты статического анализа кода](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%BA%D0%BE%D0%B4%D0%B0), которые пытаются анализируя исходный код программы обнаружить ошибки или плохие куски кода в ней, при этом не запуская саму программу непосредственно. [Mypy](http://mypy-lang.org/) --- один из таких инструментов. Применение его к тому же исходному коду не приведет к обнаружению ошибок. 
```{figure} /_static/lecture_specific/dynamic_typing/mypy_no_error.gif
```


Однако если добавить подсказку типа для параметра `x` следующим образом. 
```python
def f(x: int):
    return x + 1

s = "abc"
print(f(s))
```
```{figure} /_static/lecture_specific/dynamic_typing/mypy_error.gif
```


Тогда `mypy` обнаружит несоответствие типа параметра `x` и переменной `s`. 

В большинство современных `IDE` встроен тот или иной статический анализатор кода. Анимация ниже демонстрирует, что при добавлении аннотации типа параметра `x`, появляется индикации ошибки в строке с вызовом функции `f`. 
```{figure} /_static/lecture_specific/dynamic_typing/IDE_type_checking.gif
```

### Документирование кода

Рассмотрим функцию `integrate` на языке `С++` со следующей сигнатурой. 

```c++
double integrate(std::function<double(double)> f, double a, double b)
```

Исходя из типов параметров, легко предположить, что
1) в качестве первого параметра необходимо передать функцию $f$, которая должна принимать на вход число типа `double` и возвращать тоже число типа `double`;  
2) в качестве второго и третьего параметров ожидаются границы отрезка интегрирования в виде двух чисел типа `double`;
3) функция `integrate` возвращает результат интегрирования в виде числа типа `double`.

Т.е. сигнатура функции подсказывает пользователю, как её правильно вызывать, тем самым осуществляя документирующую функцию. Для сравнения рассмотрим, как может выглядеть сигнатура аналогичной функции в языке с динамической типизацией. 

```python
def integrate(f, a, b):
    ...
```

Смотря только на сигнатуру, можно только предположить, что `f` --- функция, а параметры `a` и `b` --- границы интегрирования. При этом только из сигнатуры не удается понять, какое поведение ожидается от функции `f`, а также в каком виде возвращается результат вычисления интеграла: число или может быть структура данных с дополнительной отладочной информацией? Схожего документирующего эффекта можно добиться следующей аннотацией функции `integrate` из предыдущего примера: приведенный ниже код корректен с точки зрения синтаксиса `python`.

```python
from collections.abc import Callable

def integrate(f: Callable[[float], float], a: float, b: float) -> float:
    ...
```

### Автозаполнение в IDE

Среды разработки стараются упростить процесс разработки и опытные программисты этим пользуются. В частности `IDE` на основе типов переменных и параметров функции могут определить список доступных методов и предложить возможные варианты для автоматического заполнения. 

```{figure} /_static/lecture_specific/dynamic_typing/cpp_hint.gif
```

В анимации выше среда разработки с каждой набранной буквой метода `push_back` подсказывает все более узкий список возможных продолжений, т.к. среде разработки известно, что переменная `x` является контейнером [std::vector](https://en.cppreference.com/w/cpp/container/vector), все методы которого ей (среде разработки) тоже известны. 

В случае с динамической типизацией наблюдается совершенно иная картина.
```{figure} /_static/lecture_specific/dynamic_typing/python_no_hint.gif
```
При вызове функции `f` на месте переменной `x` может оказаться объект любого типа. Даже если программисту известно, что это всегда будет список, у которого есть метод `append`, то среде разработки это отнюдь не очевидно. Это приводит к тому, что она не может подсказать возможное продолжение, т.к. список возможных продолжений близок к безграничному. 

Однако если подсказать среде разработке, что `x` --- переменная типа `list`, то она начнет реагировать в соответствии. 

```{figure} /_static/lecture_specific/dynamic_typing/python_hint.gif
```